In [1]:
from DrissionPage import ChromiumPage
from pprint import pprint
import csv
import json
import pandas as pd
import time  
import random 
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
f = open('job.csv', 'w', encoding='utf-8', newline='')
csv_writer = csv.DictWriter(f, fieldnames=['职位', '薪资', '学历', '经验', '公司', '城市', '领域', '规模', '性质', '区域', '商圈', '纬度', '经度', '技能列表', '福利列表'])
csv_writer.writeheader()

dp = ChromiumPage()
dp.listen.start('search/joblist.json')

# 定义关键词列表
keywords = ['python', '测试', '前端', '算法', '数据分析', '运维', '产品经理', 'UI', 'HR', '教师', '会计', '工程师']

for keyword in keywords:
    print(f'\n===== 开始采集关键词「{keyword}」 =====')
    dp.get(f'https://www.zhipin.com/web/geek/jobs?query={keyword}&city=100010000')

    for page in range(1, 21):
        print(f'正在采集第{page}页的数据内容')
        r = dp.listen.wait()
        json_data = r.response.body
        print(type(r.response.body))
        joblist = json_data['zpData']['jobList']

        for job in joblist:
            try:
                dit = {
                    '职位': job['jobName'],
                    '薪资': job['salaryDesc'],
                    '学历': job['jobDegree'],
                    '经验': job['jobExperience'],
                    '公司': job['brandName'],
                    '城市': job['cityName'],
                    '领域': job['brandIndustry'],
                    '规模': job['brandScaleName'],
                    '性质': job['brandStageName'],
                    '区域': job['areaDistrict'],
                    '商圈': job['businessDistrict'],
                    '纬度': job['gps']['latitude'],
                    '经度': job['gps']['longitude'],
                    '技能列表': ' '.join(job['skills']),
                    '福利列表': job['welfareList'],
                }
                csv_writer.writerow(dit)
                print(dit)
            except:
                pass

            tab = dp.ele('css:.job-list-container')
            dp.scroll.to_bottom()

    # 每个关键词爬取完后随机延时 10~30 秒（防止被封号）
    delay = random.uniform(10, 30)
    print(f'\n===== 关键词「{keyword}」采集完成，等待 {delay:.2f} 秒后继续 =====')
    time.sleep(delay)

f.close()

# 设置中文显示
plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False

# 读取原始数据
df = pd.read_csv('job.csv')

# 缺失值处理
df['技能列表'] = df['技能列表'].fillna('无')
df['福利列表'] = df['福利列表'].fillna('无')

# ========== 识别薪资类型 ==========

def detect_salary_type(s):
    if isinstance(s, str):
        if '元/天' in s:
            return '日薪'
        elif '元/小时' in s:
            return '时薪'
        elif 'K' in s:
            return '月薪'
    return '其他'

df['薪资类型'] = df['薪资'].apply(detect_salary_type)

# ========== 仅保留月薪职位并解析为数值 ==========

def parse_month_salary(s):
    if isinstance(s, str) and 'K' in s:
        s = s.replace('K', '')
        try:
            if '-' in s:
                low, high = map(float, s.split('-'))
                return (low + high) / 2 * 1000
            elif '以上' in s:
                return float(s.replace('以上', '')) * 1000
            elif '以下' in s:
                return float(s.replace('以下', '')) * 1000
        except:
            return None
    return None

df['平均薪资（元）'] = df['薪资'].apply(parse_month_salary)

# 筛选有效数据（仅月薪，且薪资已解析）
df_month = df[(df['薪资类型'] == '月薪') & (df['平均薪资（元）'].notnull())]

# 保存清洗后的数据
df_month.to_csv('job_clean_month_only.csv', index=False, encoding='utf-8-sig')

# ========== 描述性分析 ==========

print("🔍 有效月薪职位数量：", len(df_month))

# 平均薪资 Top10 职位
top_jobs = df_month.groupby('职位')['平均薪资（元）'].mean().sort_values(ascending=False).head(10)
print("\n💰 平均薪资Top10职位：")
print(top_jobs)

plt.figure(figsize=(10, 6))
top_jobs.plot(kind='barh', color='skyblue')
plt.title('平均薪资Top10职位')
plt.xlabel('平均薪资（元）')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# 学历分布
plt.figure(figsize=(8, 5))
sns.countplot(data=df_month, y='学历', order=df_month['学历'].value_counts().index)
plt.title('学历要求分布')
plt.tight_layout()
plt.show()

# 经验分布
plt.figure(figsize=(8, 5))
sns.countplot(data=df_month, y='经验', order=df_month['经验'].value_counts().index)
plt.title('经验要求分布')
plt.tight_layout()
plt.show()

# 城市平均薪资（如有多个城市）
plt.figure(figsize=(10, 5))
city_salary = df_month.groupby('城市')['平均薪资（元）'].mean().sort_values(ascending=False)
city_salary.plot(kind='bar', color='lightgreen')
plt.title('不同城市职位平均薪资')
plt.ylabel('元')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 福利词云
text = ' '.join(df_month['福利列表'].dropna().astype(str))
wc = WordCloud(font_path='simhei.ttf', background_color='white', width=800, height=400).generate(text)
plt.figure(figsize=(10, 5))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title("福利关键词词云")
plt.show()



KeyboardInterrupt: 